In [1]:
import numpy as np
import sys
from casadi import *

# Add do_mpc to path. This is not necessary if it was installed via pip
sys.path.append('../../../')

# Import do_mpc package:
import do_mpc

In [2]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model('continuous', 'MX')
# model = do_mpc.model.Model(model_type)

In [3]:
g = 9.80665 # m/s^2, Gravity

In [4]:
pos = model.set_variable('_x',  'pos',2)
dpos = model.set_variable('_x',  'dpos',2)


u = model.set_variable('_u',  'force',2)

ddpos = model.set_variable('_z', 'ddpos',2)
# prin

In [5]:
model.set_rhs('pos', dpos)
model.set_rhs('dpos', ddpos)
# model.set_rhs('time', 1)

In [6]:
rest = 1

d = MX.sym('x',2)

d_len = Function('f',[d],\
           [sqrt(d[0]**2 + d[1]**2)])
# print(m_len(np.array([[3],[4]])))
s = Function('f',[d],\
           [(rest - d_len(d))/d_len(d)])

print(s(np.array([[3],[4]])))

t = Function('f',[d],\
           [d * 0.5 * s(d)])

print(t(np.array([[3],[4]])))


-0.8
[-1.2, -1.6]


In [7]:
formulas = vertcat(
        # 1
        u*0.001 + [0,-g] + 10 * t(pos-[50,50])
        - (ddpos),
    )

model.set_alg('formulas', formulas)

In [8]:

distance = (50-pos[1])**2 + (50 - pos[0])**2


model.set_expression('distance', distance)

MX((sq((50-pos[1]))+sq((50-pos[0]))))

In [9]:
# Build the model
model.setup()

In [10]:
mpc = do_mpc.controller.MPC(model)

In [11]:
setup_mpc = {
    'n_horizon': 100,
    'n_robust': 0,
    'open_loop': 0,
    't_step': 0.04,
    'state_discretization': 'collocation',
    'collocation_type': 'radau',
    'collocation_deg': 3,
    'collocation_ni': 1,
    'store_full_solution': True,
    # Use MA27 linear solver in ipopt for faster calculations:
    'nlpsol_opts': {'ipopt.linear_solver': 'mumps'}
}
mpc.set_param(**setup_mpc)

In [12]:
mterm = model.aux['distance'] # terminal cost
lterm = model.aux['distance'] # stage cost

mpc.set_objective(mterm=mterm, lterm=lterm)
# Input force is implicitly restricted through the objective.
mpc.set_rterm(force=0.1)

In [13]:
mpc.bounds['lower','_u','force'] = [-20,20]
mpc.bounds['upper','_u','force'] = [20,20]

In [14]:
mpc.setup()

In [15]:
estimator = do_mpc.estimator.StateFeedback(model)

In [16]:
simulator = do_mpc.simulator.Simulator(model)

In [17]:
params_simulator = {
    # Note: cvode doesn't support DAE systems.
    'integration_tool': 'idas',
    'abstol': 1e-10,
    'reltol': 1e-10,
    't_step': 0.04
}

simulator.set_param(**params_simulator)

In [18]:
simulator.setup()

In [19]:
simulator.x0['pos'] = [20,10]



x0 = simulator.x0.cat.full()

mpc.x0 = x0
estimator.x0 = x0

mpc.set_initial_guess()


In [20]:
import matplotlib.pyplot as plt
plt.ion()
from matplotlib import rcParams
rcParams['text.usetex'] = False
rcParams['axes.grid'] = True
rcParams['lines.linewidth'] = 2.0
rcParams['axes.labelsize'] = 'xx-large'
rcParams['xtick.labelsize'] = 'xx-large'
rcParams['ytick.labelsize'] = 'xx-large'

In [21]:
mpc_graphics = do_mpc.graphics.Graphics(simulator.data)
# print(pos)

In [22]:
%%capture

fig = plt.figure(figsize=(16,9))
fig.patch.set_facecolor('white')
plt.title("space ship")

ax1 = plt.subplot2grid((4, 2), (0, 0), rowspan=4)
ax2 = plt.subplot2grid((4, 2), (0, 1))
# ax3 = plt.subplot2grid((4, 2), (1, 1))
ax4 = plt.subplot2grid((4, 2), (2, 1))
# ax5 = plt.subplot2grid((4, 2), (3, 1))

ax2.set_ylabel('Distance to goal')
# ax3.set_ylabel('Posx ')
ax4.set_ylabel('Position x,y')
# ax5.set_ylabel('Input accel x,y')

# Axis on the right.
for ax in [ax2, ax4]:
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
#     if ax != ax5:
    ax.xaxis.set_ticklabels([])

# ax5.set_xlabel('time [s]')

# ax1.axhline(0,color='black')
ax1.set_xlim(0,120)
ax1.set_ylim(0,120)

bar1 = ax1.plot([],[], '-o', linewidth=5, markersize=10)
bar2 = ax1.plot([],[], '-o', linewidth=5, markersize=10)

# mpc_graphics.add_line(var_type='_aux', var_name='d', axis=ax1)
mpc_graphics.add_line(var_type='_aux', var_name='distance', axis=ax2)
# mpc_graphics.add_line(var_type='_x', var_name='pos', axis=ax3)
mpc_graphics.add_line(var_type='_x', var_name='pos', axis=ax4)
# mpc_graphics.add_line(var_type='_u', var_name='force', axis=ax5)


fig.align_ylabels()
fig.tight_layout()

In [23]:
# u0 = mpc.make_step(x0)

In [24]:


# mpc_graphics.plot_predictions()
# mpc_graphics.reset_axes()

# fig

In [25]:
# %%capture
# Quickly reset the history of the MPC data object.
mpc.reset_history()

n_steps = 300
# u0 = mpc.make_step(x0)
# print("~~~~~")
# print(u0)
# print(type(u0))
# print("~~~~~")
for k in range(n_steps):
#     u0 = mpc.make_step(x0)
#     print("~~~~~")
#     print(u0)
#     print("~~~~~")
    y_next = simulator.make_step(np.array([[0],[0]]))
    x0 = estimator.make_step(y_next)
    

The linesearch algorithm failed with too small a step.


RuntimeError: .../casadi/interfaces/sundials/idas_interface.cpp:591: IDACalcIC returned "IDA_LINESEARCH_FAIL". Consult IDAS documentation.

In [ ]:
from matplotlib.animation import FuncAnimation, FFMpegWriter, ImageMagickWriter

# The function describing the gif:
# x = mpc.data['_x']
x = simulator.data['_x']
print(x)
# print(list(map(lambda n: [n[0]/100,n[1]/100], x)))
def update(t_ind):

    bar1[0].set_data(x[t_ind][0],x[t_ind][1])
    bar2[0].set_data(50,50)
    mpc_graphics.plot_results(t_ind)
#     mpc_graphics.plot_predictions(t_ind)
    mpc_graphics.reset_axes()


anim = FuncAnimation(fig, update, frames=n_steps, repeat=False)
gif_writer = ImageMagickWriter(fps=20)
anim.save('anim.gif', writer=gif_writer)

In [ ]:
from IPython.display import HTML, Image
display(Image(data=open("anim.gif",'rb').read(), format='png'))

In [ ]:
# %%capture
# # Quickly reset the history of the MPC data object.
# mpc.reset_history()

# n_steps = 100
# for k in range(n_steps):
#     u0 = mpc.make_step(x0)
#     y_next = simulator.make_step(u0)
#     x0 = estimator.make_step(y_next)